In [ ]:
import glob
import pandas as pd
import numpy
import matplotlib.pylab as plt
import scipy

## NITRATES

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Nitrates"

data_file_n = '/Users/' + user + '/Dropbox/CropProdLabKSU/Lab Analysis/VISIONlite Results/' + folder

glob.os.chdir(data_file_n)

AllNfiles = glob.glob(data_file_n + "/*.csv")

Concentration = []

Sample = []

columns = ['ID', 'Concentration']
Nitrate_final = pd.DataFrame(columns=columns)

for file in AllNfiles:
    nit_df = pd.read_csv(file, header=0)
    nit_df.rename(columns = {'Sample Name':'ID', '410 nm   [A]':'B'}, inplace=True)
    
    nit_df = nit_df[['ID', 'B']]

    nit_g = nit_df.groupby('ID')

    nit_final=nit_df.merge(nit_g.mean(),on='ID')
    nit_final2=nit_final.merge(abs(nit_g.max()-nit_g.min())/nit_g.max(), on='ID')
    nit_final2.rename(columns = {'B':'Var', 'B_x':'Abs', 'B_y':'Mean'}, inplace=True)
 
    c = (0,0,0,1500,1500,1500,3000,3000,3000,4500,4500,4500)
    a = nit_final2.iloc[4:16, 1]
    m = nit_final2.iloc[4:16, 2]

    nit_curve_val = nit_df.iloc[4:16, 0]
    
    try: 
        float(nit_curve_val[0:2].all()) == 0.00 
        float(nit_curve_val[3:5].all()) == 1500.00
        float(nit_curve_val[6:8].all()) == 3000.00
        float(nit_curve_val[9:11].all()) == 4500.00
    
        try:
            plt.plot(c, a, "ro")
            plt.plot(c, m,'bx')
            plt.ylabel("Absorbance Value")
            plt.xlabel("Concentration ppm")

            z = numpy.polyfit(c, m, 1, rcond=None, full=False, w=None, cov=False)
            p = numpy.poly1d(z)
            plt.plot(c, p(c),"g--")
            print("Nitrate Calibration Curve")
            print(file)
            plt.show()
            print("absorbance = %.6f(concentration) + %.6f"%(z[0],z[1]))
            print("concentration = (absorbance - %.6f) / %.6f"%(z[1],z[0]))

            x=z[1]
            y=z[0]           

            for key, row in nit_final2.iterrows():
                if float(row['Var']) > 0.1:
                    Concentration.append('Error')
                else:
                    Concentration.append(((float(row['Mean']) -x)/y).round(3))
                
                Sample.append(row['ID'])
                          
        except: ValueError
        continue
    
    except:
        print(file)
        print("Error: Unable to identify calibration curve.")
        
Nitrate_final['Concentration'] = Concentration
Nitrate_final['ID'] = Sample

## UREIDES

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = 'Ureides'

data_file_u = '/Users/' + user + '/Dropbox/CropProdLabKSU/Lab Analysis/VISIONlite Results/' + folder

glob.os.chdir(data_file_u)

AllUfiles = glob.glob(data_file_u + "/*.csv")

Concentration_u = []

Sample_u = []

columns_u = ['ID', 'Concentration']
Ureide_final = pd.DataFrame(columns=columns_u)

for file in AllUfiles:
    ur_df = pd.read_csv(file, header=0)
    ur_df.rename(columns = {'Sample Name':'ID', '535 nm   [A]':'B'}, inplace=True)
    
    ur_df = ur_df[['ID', 'B']]
    
    ur_g = ur_df.groupby('ID')

    ur_final=ur_df.merge(ur_g.mean(),on='ID')
    ur_final2=ur_final.merge(abs(ur_g.max()-ur_g.min())/ur_g.max(), on='ID')
    ur_final2.rename(columns = {'B':'Var', 'B_x':'Abs', 'B_y':'Mean'}, inplace=True)

    c_u = (0,0,0,549.6,549.6,549.6,1099.2,1099.2,1099.2,4396.7,4396.7,4396.7)
    a_u = ur_final2.iloc[4:16, 1]
    m_u = ur_final2.iloc[4:16, 2]

    ur_curve_val = ur_df.iloc[4:16, 0]
    
    try:
        float(ur_curve_val[0:2].all()) == 0.00
        float(ur_curve_val[3:5].all()) == 549.58
        float(ur_curve_val[6:8].all()) == 1099.17
        float(ur_curve_val[9:11].all()) == 4396.66
    
        try:
            plt.plot(c_u, a_u, "ro")
            plt.plot(c_u, m_u,'bx')
            plt.ylabel("Absorbance Value")
            plt.xlabel("Concentration ppm")

            z_u = numpy.polyfit(c_u, m_u, 1, rcond=None, full=False, w=None, cov=False)
            p_u = numpy.poly1d(z_u)
            plt.plot(c_u, p_u(c_u),"g--")
            print("Ureide Calibration Curve")
            print(file)
            plt.show()
            print("absorbance = %.6f(concentration) + %.6f"%(z_u[0],z_u[1]))
            print("concentration = (absorbance - %.6f) / %.6f"%(z_u[1],z_u[0]))

            x_u=z_u[1]
            y_u=z_u[0]           

            for key, row in ur_final2.iterrows():
                if float(row['Var']) > 0.1:
                    Concentration_u.append('Error')
                else:
                    Concentration_u.append(((float(row['Mean'])-x_u)/y_u).round(3))
                
                Sample_u.append(row['ID'])
        except:
            print("Curve Error")
            continue
    except:
        print(file)
        print("Error: Curve not identified.")
        
Ureide_final['Concentration'] = Concentration_u
Ureide_final['ID'] = Sample_u

In [39]:
UN_final = Ureide_final.merge(Nitrate_final, on='ID')
UN_final.rename(columns = {'Concentration_x':'Conc_Ureides', 'Concentration_y':'Conc_Nitrates'}, inplace=True)
Sorted_final = UN_final.groupby('ID')
Sorted_final.count()

,Conc_Ureides,Conc_Nitrates
ID,,
0,36,36
0.00,3933,3933
1500.0,171,171
1_1,567,567
1_2,567,567
2_1,540,540
2_2,540,540
3000.0,171,171
4500.0,171,171
